In [ ]:
# Googleドライブへのマウント（Colab用コード）
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from PIL import Image

def imshow(img):
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

In [ ]:
src = cv2.imread("prims.png", cv2.IMREAD_GRAYSCALE) 
noise = np.random.normal(0.0, 32.0, src.shape) # ノイズ付加
src = np.clip(src + noise,0,255).astype(np.uint8)
imshow(src)

In [ ]:
# 閾値128で2値化
retval, binary = cv2.threshold(src,128,255,cv2.THRESH_BINARY)
print("Binarized by", retval)
imshow(binary)

# 大津の2値化
retval, binary_otsu = cv2.threshold(src,0,255,cv2.THRESH_OTSU)
print("Otsu's thresholding: threshold=",retval)
imshow(binary_otsu)

In [ ]:
# 8近傍カーネル（8近傍と自身がすべて1）
kernel8 = np.ones((3,3),np.uint8)

# dilation
print("dilation")
dilation = cv2.dilate(binary_otsu,kernel8,iterations = 1)
imshow(dilation)

# erosion
print("erosion")
erosion = cv2.erode(binary_otsu,kernel8,iterations = 1)
imshow(erosion)

# opening
print("opening")
opening = cv2.morphologyEx(binary_otsu, cv2.MORPH_OPEN, kernel8)
imshow(opening)

# opening
print("closing")
closing = cv2.morphologyEx(binary_otsu, cv2.MORPH_CLOSE, kernel8)
imshow(closing)

# a combination of opening & closing
print("combination of opening & closing")
dst = cv2.morphologyEx(binary_otsu, cv2.MORPH_CLOSE, kernel8,iterations=2)
dst = cv2.morphologyEx(dst, cv2.MORPH_OPEN, kernel8,iterations=2)
imshow(dst)

In [ ]:
# labeling
n_labels, labels, data, center = cv2.connectedComponentsWithStats(dst)
print("[labeling] n_labels:", n_labels)

In [ ]:
# first component (0: background)
obj = ((labels==1)*255).astype(np.uint8)
imshow(obj)

# moments
M = cv2.moments(obj)
print("moments", M)

# area
print("area =",M['m00'])

# center
cx = (M['m10']/M['m00'])
cy = (M['m01']/M['m00'])
print("center = (",cx,",",cy,")")

# 主軸を求めてみよう（提出不要）



参考

https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html

https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html